Python 中的对象属性本质上是公共的，并且不支持任何访问说明符来使它们成为私有或受保护的。您不能阻止任何调用对象访问基础对象上的任何属性。
Python 没有严格执行访问说明符，但它是一种社区驱动的语言，有几个约定可以弥补它。当你用双下划线开始一个属性时，你表明它对于对象的范围是私有的。

## 可迭代对象及其创建

Python 中

```python
for element in elements:
    ...

```

1. 会判断对象是否具有 __iter__, 或 __next__ 迭代器方法 
2. 被迭代的序列对象是否具有 __getitem__ 或 __len__ 方法

In [26]:
class Fibonacci:
    def __init__(self, n):
        self.n = n
        self.previous = 0
        self.current = 1
        self._count = 0

    def __iter__(self):
        print("use iter")
        return self
    
    def __next__(self):
        if self._count > self.n:
            raise StopIteration
        
        self._count += 1
        return_value = self.previous
        self.previous, self.current = self.current, self.previous + self.current
        return return_value


class tmp_iter:
    def __init__(self, n):
        self.n = n
        self.previous = 0
        self.current = 1
        self._count = 0

    def __iter__(self):
        return Fibonacci(10)
    
    # def __next__(self):
    #     if self._count > 10:
    #                 raise StopIteration
    #     return 1
    #     self._count += 1

In [31]:
# for num in Fibonacci(10):
#     print(num, end = " ")

for num in tmp_iter(10):
    print(num, end = " ")

0 1 1 2 3 5 8 13 21 34 55 

In [4]:
t = Fibonacci(10)
for num in t:
    print(num, end = ', ')

print()
for num in t:
    print(num, end = ', ')

use iter
0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 
use iter


**对上述结果的一些分析**：

首先我们创建了一个, Fibonacci 类, 这个类具有 ```__iter__``` 和 ```__next__```属性

在使用 for 循环迭代时会调用 iter, 返回一个可迭代对象(具有 next) 方法。

```python
t = Fibonacci(10)
```

只创建了一次，在第一个 for 循环中迭代完成后，内部已经完成了数据的生成。

第二个 for 循环不会再生成新的数据。

In [12]:
def fib(n):
    prev, current = 0, 1
    for _ in range(n + 1):
        yield prev
        prev, current = current, prev + current

In [ ]:
for num in fib(5):
    print(num, end = " ")

## 序列的创建
确保实现了
```python
__len__()
```

和
```python
__gettiem__()
```

方法

### 与迭代器对比
会占用更多的内存，但是获取数据的时间会更快。同时支持随机访问。

本质上是一种 时间和空间 的交换。

In [28]:
from datetime import date, timedelta

In [29]:
class DataRange:
    def __init__(self, start_dt, end_dt):
        self.start_dt = start_dt
        self.end_dt = end_dt
        self._range_values = self._get_range_values()

    def _get_range_values(self):
        data = []

        current_dt = self.start_dt
        while current_dt < self.end_dt:
            data.append(current_dt)
            current_dt += timedelta(days=1)

        return data
    
    def __len__(self):
        return len(self._range_values)
    
    def __getitem__(self, index):
        return self._range_values[index]

In [30]:
my_data_range = DataRange(date(2022, 6, 1), date(2022, 6, 10))
for my_data in my_data_range:
    print(my_data)

2022-06-01
2022-06-02
2022-06-03
2022-06-04
2022-06-05
2022-06-06
2022-06-07
2022-06-08
2022-06-09


## 容器对象
Python 中实现 ```__contains__()``` 魔术方法并返回真值的对象称为容器。它通常与 in 运算符一起使用以检查成员资格。
容器是包含任意数量的其他对象的对象。通常，容器提供了一种访问所包含对象并对其进行迭代的方法。
Python内建的容器包括 tuple、list、set、dict等。在实践中，所有容器都将具有 ```__contains__``` 魔术方法。

测试一个对象是否是一个容器时，应该使用:
```python
isinstance(x, collections.abc.Container) 
```

另外，从面向对象与代码维护的角度看，容器对象用好了，可以写出容易维护的代码。


**鸭子方法:**: 看起来像鸭子，跑起来像鸭子。那它就是鸭子。

In [41]:
class Product:
    def __init__(self, name:str, price:float, spec_num: int) -> None:
        self.name = name
        self.price = price
        self.spec_num = spec_num    # 根据编号决定使用那种促销

class Promotion:
    def __init__(self, lower_num:int, upper_num: int, rate: float) -> None:
        self.__lower_num = lower_num
        self.__upper_num = upper_num 
        self.__rate = rate

    @property
    def rate(self) -> float:
         return self.__rate
    
    def __contains__(self, product: Product) -> bool:
        return self.__lower_num <= product.spec_num <= self.__upper_num
    
from typing import List

def get_total_price(products: List[Product], promotions: List[Promotion]) -> float:
    total_price = 0

    for t in promotions:
        pass

    for product in products:
        promotion = [promotion for promotion in promotions if product in promotion][0]
        total_price += product.price * promotion.rate

    return total_price

In [42]:
top_promotion = Promotion(100, 199, 0.5)
average_promotion = Promotion(50, 99, 0.8)
none_promotion = Promotion(0, 49, 1.0)
promotions = (top_promotion, average_promotion, none_promotion)

products = (
    Product('cart', 1999.9, 188),
    Product('computer', 5999.9, 88),
    Product('toy', 22.5, 33)
)

total_price = get_total_price(products, promotions)
print(total_price)
print(1999.9 * 0.5 + 5999.9 * 0.8 + 22.5 * 1)

5822.37
5822.37


In [44]:
import collections
import collections.abc

a = [1, 2, 3]
b = {'a': 1, 'c': 2}
c = (1, 2, 3)

print(isinstance(average_promotion, collections.abc.Container))
print(isinstance(a, collections.abc.Container))
print(isinstance(b, collections.abc.Container))
print(isinstance(c, collections.abc.Container))

True
True
True
True


## 动态处理对象属性

Python 中访问一个类的任何属性都会经过 ```__getattribute__``` 方法, 包括类本身包含(编写该类时定义的属性，以及一些默认属性：例如 ```__dict__``` )的属性和实例化后新增的属性。

可以通过重构 ```__getattribute__``` 方法, 在获取对象属性时，对属性做一些中间操作。（一般情况下不建议这么做，因为这样不符合单一功能原则，同时可读性也会较差）

对于访问不存在的属性，回进入 ```__getattr__``` 方法，在这里可以处理不存在的属性，返回一些特定值，或者抛出异常进行提示。

__getattribute__是属性访问拦截器，就是当这个类的属性被实例访问时，会自动调用类的__getattribute__方法。

可以看到下面的代码在访问 name, ```__dict__``` 这些默认属性和新添加的属性 age 时都会经过 ```__getattribute__``` 方法而不会经过 ```__get_attr__``` 方法。

对于访问不存在的属性 nick_name，则会同时经过 ```__getattr__``` 和 ```__getattribute__``` 方法。

In [1]:
class Person:
    def __init__(self, name: str) -> None:
        self.name = name

    def __getattr__(self, attr):
        print('** get attr **', attr)
        return f'Not support attribute "{attr}".'

    def __getattribute__(self, attr):
        print('** get attribute **', attr)
        # raise AttributeError(f'Not support attribute "{attr}".')
        return super().__getattribute__(attr)

In [2]:
person = Person('Steven')
person.age = 23
# print(person.name, person.age)
print(person.name, person.age, person.nick_name)
print(person.__dict__)

** get attribute ** name
** get attribute ** age
** get attribute ** nick_name
** get attr ** nick_name
Steven 23 Not support attribute "nick_name".
** get attribute ** __dict__
{'name': 'Steven', 'age': 23}


```__getattribute__```方法的一些使用场景

在访问某些属性前对其权限进行判断

In [3]:
from typing import List

class Context:
    def __init__(self, user: str, roles: List[str] = []) -> None:
        self.user = user
        self.roles = set(roles)

    def has_permission(self, role: str) -> bool:
        return role in self.roles
    
    def grant_permission(self, role: str) -> None:
        self.roles.add(role)

class Project:
    def __init__(self, name: str, price: float, context: Context) -> None:
        self.name = name
        self.price = price
        self.context = context

    def __getattribute__(self, attr):
        if attr == 'price':
            if self.context.has_permission('owner'):
                return super().__getattribute__(attr)
            else:
                raise AttributeError(f'访问属性 “{attr}”失败.')
        return super().__getattribute__(attr)

In [4]:
context = Context('Steven', ['user', 'admin'])
project = Project('Big Project', 1680000.0, context)
context.grant_permission('owner')
print(project.price)

1680000.0


## 可调用对象

Python 允许您创建行为类似于函数的对象。此功能的众多应用之一是创建更健壮的装饰器。当试图执行对象时，我们使用__call__ 魔术方法l来模拟正常函数。此外，提供给对象的参数将被移交给 __call__ 方法。

__call__方法是双刃剑，用好了让你的代码丝般顺滑，让人容易理解，容易维护；但用的不好的话，会导致羞涩难懂的尴尬。它作为后续装饰器decorator的铺垫起到了重要的作用。



In [9]:
class Person:
    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

    def __call__(self, weight):
        self.weight = weight
        print(self.name, self.age, self.weight)
        return self

In [15]:
person = Person('Steven', 23)
print(callable(person))
its_me = person(100)
print(its_me.name, its_me.age, its_me.weight)

curme = its_me(200)
print(curme.name, curme.age, curme.weight)

print('****************')
print(person.name, person.age, person.weight)
print(its_me.name, its_me.age, its_me.weight)
print(curme.name, curme.age, curme.weight)

True
Steven 23 100
Steven 23 100
Steven 23 200
Steven 23 200
****************
Steven 23 200
Steven 23 200
Steven 23 200


In [ ]:
# 可记录函数状态的，函数调用跟踪器
# 下面的代码相当于一个函数调用计数器

from collections import defaultdict

class TrackFuntionCalls:
    def __init__(self):
        self._call_count = defaultdict(int)

    def __call__(self, arg):
        self._call_count[arg] += 1
        return self._call_count[arg]


In [27]:
call_count = TrackFuntionCalls()
call_count(2022)

1

In [28]:
call_count(1988)

1

In [29]:
call_count(2022)

2

In [30]:
call_count(9911)

1

## \_\_str__ 和 \_\_repr\_\_

当你定义一个你知道你会在某个时候 print() 的类时，默认的 Python 表示可能不是很有帮助。您可以定义自定义 ```__str__``` 方法来控制不同组件的打印方式，以获得更好的清晰度和可读性。
```__str__``` 方法实现允许我们以人类可读的方式显示类，```__repr__``` 方法可用于获取类的机器可读格式。
Python 类中__repr__ 的事实上的实现没有重要用途。该方法应该具有构造对象所需的一切。对象和 ```__repr__``` 输出应该是可区分的对象。这在日志记录用例中很有帮助；例如，列表对象的组成部分使用 ```__repr__``` 而不是 ```__str__``` 表示形式显示。

In [48]:
class Person:
    def __init__(self, name: str, interests:List[str]) -> None:
        self.name = name
        self.interests = interests

    def __str__(self):
        return f'{self.name} like {",".join(self.interests)}'
    
    def __repr__(self):
        return f'Person(name="{self.name}", interests={self.interests})'
    
    def __eq__(self, other):
        return self.name == other.name and self.interests == other.interests

In [51]:
person = Person('Steven', ['football', 'basketball', 'swimming'])
print(person)
str(person)
repr(person)

t = eval(repr(person))

t == person

Steven like football,basketball,swimming


True

In [53]:
# 一般而言，str 是给用户看的。repr 是给开发者看的

s = 'a"b'
str(s)
repr(s)
print(str(s), repr(s))

a"b 'a"b'


In [55]:
import datetime

print(datetime.datetime.now())

2025-02-10 13:42:59.434354


In [57]:
print(repr(datetime.datetime.now()))

datetime.datetime(2025, 2, 10, 13, 43, 35, 299201)
